In [1]:
import numpy as np
import seaborn as sn
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator( rescale = 1./255,
                                    validation_split=0.2,
                                    rotation_range = 20,
                                    horizontal_flip = True,
                                    vertical_flip=True,
                                    fill_mode = 'nearest'
                                  )

val_datagen = ImageDataGenerator(rescale = 1./255, 
                                 validation_split=0.2,
                                )
test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:
input_shape = (224,224)
src = 'data'
# src = 'data' is for fer2013 dataset
# src = 'ckplus' is for CK+ dataset
# file structure should be like this
# src/train/(folders for each emotion)
# src/test/(folders for each emotion)


train_data = train_datagen.flow_from_directory(src+'/train',
                                                target_size = input_shape,
                                                class_mode = 'categorical',
                                                batch_size = 64,
                                                subset='training'
                                              )

val_data = val_datagen.flow_from_directory(src+'/train',
                                            target_size = input_shape,
                                            class_mode = 'categorical',
                                            batch_size = 64,
                                            subset='validation'
                                          )

test_data = test_datagen.flow_from_directory(src+'/test',
                                              target_size = input_shape,
                                              class_mode = 'categorical',
                                              batch_size = 64
                                            )

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
train_steps = train_data.n//train_data.batch_size
val_steps = val_data.n//val_data.batch_size
test_steps = test_data.n//test_data.batch_size

In [5]:
# VGG16 model
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(input_shape[0], input_shape[1], 3))

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False
    
# un-freeze the last 3 layers
for layer in base_model.layers[-3:]:
    layer.trainable = True
    
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # Assuming 7 classes for emotions

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [6]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
checkpoint = ModelCheckpoint('emotion_model.h5', save_best_only=True)

# Training the model
history = model.fit(train_data,
                    steps_per_epoch=train_steps,
                    epochs=100,
                    validation_data=val_data,
                    validation_steps=val_steps,
                    callbacks=[reduce_lr, checkpoint]
                   )

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_data, steps=test_steps)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')



Epoch 1/100
358/358 [==============================] - ETA: 0s - loss: 1.8558 - accuracy: 0.3055

C:\Users\samia\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


358/358 [==============================] - 411s 1s/step - loss: 1.8558 - accuracy: 0.3055 - val_loss: 1.6806 - val_accuracy: 0.3538 - lr: 0.0010
Epoch 2/100
358/358 [==============================] - 393s 1s/step - loss: 1.5911 - accuracy: 0.3826 - val_loss: 1.5313 - val_accuracy: 0.4041 - lr: 0.0010
Epoch 3/100
358/358 [==============================] - 390s 1s/step - loss: 1.5314 - accuracy: 0.4064 - val_loss: 1.5537 - val_accuracy: 0.4020 - lr: 0.0010
Epoch 4/100
358/358 [==============================] - 392s 1s/step - loss: 1.4917 - accuracy: 0.4269 - val_loss: 1.4849 - val_accuracy: 0.4250 - lr: 0.0010
Epoch 5/100
358/358 [==============================] - 409s 1s/step - loss: 1.4669 - accuracy: 0.4363 - val_loss: 1.5103 - val_accuracy: 0.4052 - lr: 0.0010
Epoch 6/100
358/358 [==============================] - 417s 1s/step - loss: 1.4453 - accuracy: 0.4484 - val_loss: 1.4881 - val_accuracy: 0.4273 - lr: 0.0010
Epoch 7/100
358/358 [==============================] - 385s 1s/step - 